*To be run on Sage kernel, with eel installed*

In [1]:
from hourglass import App

# retreive example data for cons tructor
from examples import example_2, example_1, benzene_debug, benzene_and_square
g = example_2()

app = App(graph=g, use_notebook=False)

app.editor()

App open @ http://localhost:8000/draw.html
Editor Closed.


In [2]:
# app.editor()
app2 = App(graph=g, use_notebook=False)

In [3]:
app2.editor()

App open @ http://localhost:8000/draw.html


Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 908, in gevent._gevent_cgreenlet.Greenlet.run
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/eel/__init__.py", line 191, in run_lambda
    btl.run(
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/bottle.py", line 3175, in run
    server.run(app)
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/bottle_websocket/server.py", line 17, in run
    server.serve_forever()
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/gevent/baseserver.py", line 398, in serve_forever
    self.start()
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/gevent/baseserver.py", line 336, in start
    self.init_socket()
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/gevent/pywsgi.py", line 1545, in init_socket
    StreamServer.init_socket(self)
  File "/Users/dutchhansen/.sage/local/lib/python3.11/site-packages/gevent/server.py

KeyboardInterrupt: 

In [ ]:
g = app.conv_to_sage()
g.show()